# Extract columns from source with type

The main goal of this notebook is to retrieve all column names (original) get data type and then set it to a csv file. 

I will also indicate which column contains protected attributes : [list of protected attributes](https://www.fairwork.gov.au/employee-entitlements/protections-at-work/protection-from-discrimination-at-work).

## Load packages

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# Custom helper package : https://github.com/Nathanlauga/MLHelper
from MLHelper.data.transform import convert_date_columns

In [2]:
PROJECT_PATH = '..'

## Datasets details

You need to fill the `data_informations.csv` file to complete this step.

In [3]:
df_details = pd.read_csv(f'{PROJECT_PATH}/01_collect/data_informations.csv')

In [4]:
df_details.T

,0
name,adult
source,"""https://archive.ics.uci.edu/ml/datasets/adult"""
description,"""Extraction was done by Barry Becker from the..."


## Load data

In [5]:
dfs = {}
for idx, row in df_details.iterrows():
    file = row['name']
    if '.csv' not in file: 
        file = file+'.csv'
    dfs[file[:-4]] = pd.read_csv(f'{PROJECT_PATH}/_data/{file}')
    
    display(Markdown(f'#### {file}, {dfs[file[:-4]].shape}'))
    display(Markdown(row['description']))

#### adult.csv, (48842, 15)

 "Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0)). Prediction task is to determine whether a person makes over 50K a year. "

## Convert date column

In [6]:
display(Markdown('From MLHelper custom package : `convert_date_columns`'))
help(convert_date_columns)

From MLHelper custom package : `convert_date_columns`

Help on function convert_date_columns in module MLHelper.data.transform.transform:

convert_date_columns(df)
    Automatic convert date columns as object to datetime.
    It follows the next rules:
    * Is detect as an object column
    * Don't return errors into the pandas.to_datetime function
    
    Parameters
    ----------
    df: pandas.DataFrame
        Dataframe to update
        
    Returns
    -------
    pandas.DataFrame:
        Dataframe with date column as datetime type



In [7]:
for dataset in dfs:
    dfs[dataset] = convert_date_columns(dfs[dataset]) 

## Check data type

In [8]:
for dataset in dfs:
    display(Markdown(f'**{dataset}**'))
    dtypes = dfs[dataset].dtypes
    display(dtypes)
del dtypes

**adult**

age                 int64
workclass          object
fnlwgt              int64
education          object
educational-num     int64
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income             object
dtype: object

## Display datasets head

In [9]:
for dataset in dfs:
    display(dfs[dataset].head()) 

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


## Create columns information dataset

In this part we extract all columns with their datatype to create a dataset that will be saved as `columns_informations.csv`

In [10]:
columns_df = list()

for dataset in dfs:
    dtypes = pd.DataFrame(dfs[dataset].dtypes)
    dtypes['dataset'] = dataset
    columns_df.append(dtypes)
    
del dtypes

In [11]:
columns_df = pd.concat(columns_df).reset_index()
columns_df.columns = ['column','dtype','dataset']
columns_df = columns_df[['dataset','column','dtype']]
columns_df['is_protected'] = 0

In [12]:
columns_df

,dataset,column,dtype,is_protected
0,adult,age,int64,0
1,adult,workclass,object,0
2,adult,fnlwgt,int64,0
3,adult,education,object,0
4,adult,educational-num,int64,0
5,adult,marital-status,object,0
6,adult,occupation,object,0
7,adult,relationship,object,0
8,adult,race,object,0
9,adult,gender,object,0


## Indicate protected attributes

Now, we have to indicate where are the protected attributes.

In [13]:
def set_protected_attribute(df, dataset, column):
    """
    This function return an inputed dataset (with columns informations) and 
    set the value of 1 on `is_protected` column for a specific dataset & column
    
    Parameters
    ----------
    df: pd.DataFrame
        Columns informations dataframe
    dataset: str
        dataset name
    column: str
        column name
        
    Returns
    -------
    pd.DataFrame
        Updated columns informations dataframe
    """
    df['is_protected'] = np.where((df['dataset'] == dataset) & (df['column'] == column),
                                  1,
                                  df['is_protected']
                                 )
    return df

#### What are protected attributes ?

    race
    colour
    sex
    sexual orientation
    age
    physical or mental disability
    marital status
    family or carer's responsibilities
    pregnancy
    religion
    political opinion
    national extraction
    social origin

In [14]:
columns_df = set_protected_attribute(df=columns_df, dataset='adult', column='age') 
columns_df = set_protected_attribute(df=columns_df, dataset='adult', column='marital-status') 
columns_df = set_protected_attribute(df=columns_df, dataset='adult', column='race') 
columns_df = set_protected_attribute(df=columns_df, dataset='adult', column='gender')
columns_df = set_protected_attribute(df=columns_df, dataset='adult', column='native-country')

## Finish dataset & save it

In [15]:
# Set up new name column to complete it after saving column dataset
columns_df['new_name'] = '' 
# Set up description column to complete it after saving column dataset
columns_df['description'] = '' 

In [16]:
columns_df

,dataset,column,dtype,is_protected,description,new_name
0,adult,age,int64,1,,
1,adult,workclass,object,0,,
2,adult,fnlwgt,int64,0,,
3,adult,education,object,0,,
4,adult,educational-num,int64,0,,
5,adult,marital-status,object,1,,
6,adult,occupation,object,0,,
7,adult,relationship,object,0,,
8,adult,race,object,1,,
9,adult,gender,object,1,,


In [17]:
columns_df.to_csv(f'{PROJECT_PATH}/01_collect/columns_informations.csv', index=False)

## The end

Thanks for reading,
*Nathan*